The sigmoid function maps any real-valued input to a value between 0 and 1, making it useful for binary classification in neural networks.
Apply it at the output layer when the target is 0 or 1, to interpret output as probability.
It’s used in problems like disease prediction, spam detection, etc.
Avoid it in hidden layers, as it causes vanishing gradients and slows learning.
Use ReLU or tanh in hidden layers instead for better gradient flow and training speed.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

The Breast Cancer Wisconsin dataset contains features computed from breast mass cell images to classify tumors as benign (0) or malignant (1).
It has 569 samples with 30 numerical features like radius, texture, smoothness, etc.
It’s commonly used for binary classification tasks to evaluate models like logistic regression and neural networks.

In [14]:
# Load Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [15]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 16)  # Input layer to hidden layer
        self.fc2 = nn.Linear(16, 1)                  # Hidden layer to output

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))  # Sigmoid activation
        x = self.fc2(x)                  # Output layer
        return x

In [16]:
model = SimpleNN()
criterion = nn.BCEWithLogitsLoss()  # Loss function for binary classification
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
for epoch in range(1000):
    model.train()                      # Set model to training mode
    optimizer.zero_grad()              # Clear gradients
    outputs = model(X_train_tensor)    # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Compute loss
    loss.backward()                    # Backpropagation
    optimizer.step()                   # Update weights

In [18]:
model.eval()  # Switch to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predicted = torch.sigmoid(test_outputs)  # Apply sigmoid to get probabilities
    predicted_classes = (predicted > 0.5).float()  # Apply threshold
    accuracy = (predicted_classes.view(-1) == torch.tensor(y_test, dtype=torch.float32)).float().mean()  # Calculate accuracy
    print("Predicted classes:", predicted_classes.flatten())
    print("Accuracy:", accuracy.item())

Predicted classes: tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 0.])
Accuracy: 0.9561403393745422
